In [1]:
import tensorflow as tf
import numpy as np
int2binary={}
binary_dim=8

largest_num=pow(2,binary_dim)

binary=np.unpackbits(
    np.array([range(largest_num)],dtype=np.uint8).T,axis=1)
for i in range(largest_num):
    int2binary[i]=binary[i]

def binary_generation(numbers,reverse=False):
    '''如果reverse=True,那么训练会比较方便，输入从低位开始'''
    x=np.array([int2binary[num] for num in numbers], dtype=np.uint8)
    
    if reverse:
        x=np.fliplr(x)
    return x

def batch_generation(batch_size,largest_num):
    '''生成batch_size大小的数进行验证或训练'''
    n1=np.random.randint(0,largest_num//2,batch_size)
    n2=np.random.randint(0,largest_num//2,batch_size)
    add=n1+n2
    
    #int to binary
    binary_n1=binary_generation(n1,True)
    binary_n2=binary_generation(n2,True)
    batch_y=binary_generation(add,True)
    
    #利用第三维进行深入反正合成了三维数组，
    batch_x=np.dstack((binary_n1,binary_n2))
    return batch_x,batch_y,n1,n2,add

def binary2int(binary_array):
    out=0
    for index,x in enumerate(reversed(binary_array)):
        out+=x*pow(2,index)
    return out

In [2]:
batch_size=64
lstm_size=20
lstm_layers=2


In [3]:
x=tf.placeholder(tf.float32,[None,binary_dim,2],name='input_x')
y_=tf.placeholder(tf.float32,[None,binary_dim],name='input_y')

#dropout 参数
keep_prob=tf.placeholder(tf.float32,name='keep_prob')

In [4]:
#搭建的lstm层（就是隐层）
lstm=tf.contrib.rnn.BasicLSTMCell(lstm_size)
'''
    作用：构建一个cell,选择basic...的原因是目前够用了，参考的说还有其他的
    参数：
    num_uints:LSTM cell的units数目
    forget_bias:float，遗忘门的偏置值
    state_is_tuple=True,接收和返回的是state（c_state,m_state（ht)二元组；false的话，两个连成列向量
    activation激活函数，不用多说
    reuse：是否在现有的作用域里边重用
'''

#dropout
drop=tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
'''
    作用：
    参数:用于正则化，作用是不是不同时刻，而是同一时刻的输入输出有所舍弃
    cell:循环网络的cell
    in_keep_prob:使用dropout的概率，如果为1则不执行
    output_keep_prob同理
    state_keep_prob
    variational_recurrent:bool, True的话，应用相同的模式进行所有timestep的run调用，设置的前提是有input_size值
    input_size: ？？？传进来的深度required ands used if variational_recurrent = True and input_keep_prob < 1.
    dtype:Required and used iff variational_recurrent = True.
    seed:
    dropout_state_filter_visitor:返回描述要丢弃的terms
'''
def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(lstm_size)
cell=tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
'''堆叠多个cell,但是以完整地cell形式存在，每个小cell状态会保留'''

#初始状态，state_size在上一个函数就已经创建好了，但是batch_size未知，自己给
#state是cell一依次接收num_steps个输入之后产生的
initial_state=cell.zero_state(batch_size,tf.float32)

#进行forward，得到隐层输出
#output size=[batch_size,lstm_size*binasy_dim]
outputs,final_state=tf.nn.dynamic_rnn(cell,x,initial_state=initial_state)
'''构建rnn的函数之一,输入不是tensors的list而是一份tensor
    输出pair(outputs,state)
    参数：
    cell:
    inputs:就是tensor
    squence_length=None,required, 可以动态计算
    initial_state=None
    dtype=None
    parallel_iterations=None
    swap_memory=false
    time_major=False
    scope=None
'''

weights=tf.Variable(tf.truncated_normal([lstm_size,1],stddev=0.01))
'''截断2*stddev以外的值'''
biases=tf.zeros([1])

# [batch_size,lstm_size*binary_dim]==>[batch_size*binary_dim,lstm_size]
outputs=tf.reshape(outputs,[-1,lstm_size])
#得到输出，logits大小为[batch_size*binary_size,1]
logits=tf.sigmoid(tf.matmul(outputs,weights))
#[batch_size*binary_dim,1]=>batch_size,binary_dim]
predictions=tf.reshape(logits,[-1,binary_dim])

In [5]:
cost=tf.losses.mean_squared_error(y_,predictions)
optimizer=tf.train.AdamOptimizer().minimize(cost)


In [6]:
steps=2000
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    iteration=1
    for i in range(steps):
        input_x,input_y,_,_,_=batch_generation(batch_size,largest_num)
        _,loss=sess.run([optimizer,cost],feed_dict={x:input_x,y_:input_y,keep_prob:0.5})
        if iteration%1000==0:
            print("Iter:{},Loss:{}".format(iteration,loss))
        iteration+=1
        
    #testing
    val_x,val_y,n1,n2,add=batch_generation(batch_size,largest_num)
    result=sess.run(predictions,feed_dict={x:val_x,y_:val_y,keep_prob:1.0})
    result=np.fliplr(np.round(result))
    result=result.astype(np.int32)
    
    for b_x,b_p,a,b,add in zip(np.fliplr(val_x),result,n1,n2,add):
        '''打包成元组，返回对象'''
        print('{}:{}'.format(b_x[:,0],a))
        print('{}:{}'.format(b_x[:,1],b))
        print('{}:{}\n'.format(b_p,binary2int(b_p)))

Iter:1000,Loss:0.05543878674507141
Iter:2000,Loss:0.0016524838283658028
[0 1 0 1 0 0 0 0]:80
[0 1 1 0 1 1 1 1]:111
[1 0 1 1 1 1 1 1]:191

[0 0 0 0 1 1 1 0]:14
[0 0 1 0 0 0 1 0]:34
[0 0 1 1 0 0 0 0]:48

[0 1 0 0 0 1 0 0]:68
[0 0 0 1 0 1 1 0]:22
[0 1 0 1 1 0 1 0]:90

[0 0 0 0 0 1 0 0]:4
[0 1 1 1 0 0 1 0]:114
[0 1 1 1 0 1 1 0]:118

[0 0 0 0 0 0 0 0]:0
[0 0 1 1 1 0 0 1]:57
[0 0 1 1 1 0 0 1]:57

[0 1 0 0 0 1 1 1]:71
[0 0 0 1 1 1 0 0]:28
[0 1 1 0 0 0 1 1]:99

[0 0 0 1 0 0 0 0]:16
[0 0 1 1 1 1 1 1]:63
[0 1 0 0 1 1 1 1]:79

[0 0 1 1 1 0 0 0]:56
[0 0 0 0 0 0 0 0]:0
[0 0 1 1 1 0 0 0]:56

[0 1 1 1 1 1 1 0]:126
[0 1 1 0 0 1 1 1]:103
[1 1 1 0 0 1 0 1]:229

[0 1 0 1 0 1 1 1]:87
[0 1 0 0 0 1 0 0]:68
[1 0 0 1 1 0 1 1]:155

[0 0 1 0 0 1 1 1]:39
[0 1 0 1 0 0 1 1]:83
[0 1 1 1 1 0 1 0]:122

[0 0 0 0 0 0 1 1]:3
[0 0 0 0 0 0 0 1]:1
[0 0 0 0 0 1 0 0]:4

[0 0 1 0 0 1 0 1]:37
[0 0 1 0 0 1 1 0]:38
[0 1 0 0 1 0 1 1]:75

[0 1 0 1 1 0 1 0]:90
[0 0 1 1 0 1 0 1]:53
[1 0 0 0 1 1 1 1]:143

[0 1 1 0 1 1 1 1]:111
[0 0 0